In [2]:
import warnings
warnings.filterwarnings('ignore')
import os, datetime

import numpy as np

import torch
import torchvision
import matplotlib.pyplot as plt

from torch import nn
from torch.autograd import Variable
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if cuda else 'cpu')

w, h = 224, 224 
num_filter = 128//2
num_channel = 1

def load_data(BATCH_SIZE=32):
    
    transform = transforms.Compose([
        transforms.Resize(size=(w,h)),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    Data = datasets.ImageFolder('D:\projects\NIR\data\images_for_cnn', transform=transform)
   

    batch_size = BATCH_SIZE
    test_size = 0.4
    
    num_data = len(Data)
    indices_data = list(range(num_data))
    np.random.shuffle(indices_data)
    split_tt = int(np.floor(test_size * num_data))
    train_idx, test_idx = indices_data[split_tt:], indices_data[:split_tt]
    
   
    # define samplers for obtaining training and validation batches
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    
    trainloader = torch.utils.data.DataLoader(Data, sampler = train_sampler, batch_size=batch_size, num_workers=4)
    testloader  = torch.utils.data.DataLoader(Data, sampler = test_sampler,  batch_size=batch_size, num_workers=4)
    
    return trainloader, testloader

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    
    
def noise_input(images):
    result = (images + images * torch.rand(images.size()))
    result_max = Variable(torch.max(result))
    return result / result_max#(images + images * torch.rand(images.size())
    #return images + torch.rand(images.size())
    #return images * (1 - NOISE_RATIO) + torch.rand(images.size()) * NOISE_RATIO

class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.sequential = nn.Sequential(
            nn.Conv2d(1, num_filter, 3, padding=1),
            nn.BatchNorm2d(num_filter),
            nn.LeakyReLU(0.25, inplace=True),
            nn.Conv2d(num_filter, num_filter, 3, padding=1),
            nn.BatchNorm2d(num_filter),
            nn.LeakyReLU(0.25, inplace=True),
            nn.Conv2d(num_filter, num_filter, 3, padding=1),
            nn.BatchNorm2d(num_filter),
            nn.LeakyReLU(0.25, inplace=True),
            nn.Conv2d(num_filter, num_channel, 3, padding=1),
            nn.BatchNorm2d(1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

def model_training(mycnn, train_loader, epoch):
    loss_metric = nn.MSELoss()
    optimizer = torch.optim.Adam(mycnn.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

    mycnn.train()
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        images, _ = data
        images = Variable(images)
        noise_images = noise_input(images)      # add noise into image data
        if cuda: 
            images, noise_images = images.to(device), noise_images.to(device)
        noise = mycnn(noise_images)
        outputs = noise_images / (1 - noise)
        #outputs = noise_images - noise
        loss = loss_metric(outputs, images)
        loss.backward()
        optimizer.step()
        if (i + 1) % LOG_INTERVAL == 0:
            print('Epoch [{}/{}] - Iter[{}/{}], MSE loss:{:.4f}'.format(
                epoch + 1, EPOCHS, i + 1, len(train_loader.dataset) // BATCH_SIZE, loss.item()
            ))
   #     torch.cuda.empty_cache()

def evaluation(mycnn, test_loader):
    total_loss = 0
    loss_metric = nn.MSELoss()
    mycnn.eval()
    for i, data in enumerate(test_loader):
        images, _ = data
        images = Variable(images)
        if cuda: 
            images = images.to(device)
        outputs = mycnn(images)
        loss = loss_metric(outputs, images)
        total_loss += loss.item() * len(images)
    avg_loss = total_loss / len(test_loader.dataset)

    print('\nAverage MSE Loss on Test set: {:.4f}'.format(avg_loss))

    global BEST_VAL
    if TRAIN and avg_loss < BEST_VAL:
        BEST_VAL = avg_loss
        torch.save(mycnn.state_dict(), '/content/drive/My Drive/history/conv_filter.pt')
        print('Save Best Model in HISTORY\n')

if __name__ == '__main__':
    EPOCHS = 10
    BATCH_SIZE = 8
    LEARNING_RATE = 1e-3
    WEIGHT_DECAY = 1e-5
    LOG_INTERVAL = 10
    NOISE_RATIO = 0.5
    TRAIN = False       # whether to train a model from scratch
    BEST_VAL = float('inf')     # record the best val loss

   # train_loader, test_loader = data_utils.load_mnist(BATCH_SIZE)
    
    train_loader, test_loader = load_data(BATCH_SIZE)
    mycnn = MyCNN()
    if cuda: mycnn.to(device)

    if TRAIN:
        for epoch in range(EPOCHS):
            starttime = datetime.datetime.now()
            model_training(mycnn, train_loader, epoch)
            endtime = datetime.datetime.now()
            
            print(f'Train a epoch in {(endtime - starttime).seconds} seconds')
            # evaluate on test set and save best model
            evaluation(mycnn, test_loader)
        print('Trainig Complete with best validation loss {:.4f}'.format(BEST_VAL))
    else:
        mycnn.load_state_dict(torch.load('/content/drive/My Drive/history/conv_filter.pt'))
        evaluation(mycnn, test_loader)
        
        mycnn.cpu()
        dataiter = iter(train_loader)
        images, _ = next(dataiter)
        images = Variable(images[:1])

        noise_images = noise_input(images)
        noise = mycnn(noise_images)
        outputs = noise_images / (1-noise)
        #outputs = noise_images - noise

        # plot and save original and reconstruction images for comparisons
        plt.figure(figsize=(20, 15))
        plt.subplot(131)
        plt.title('Image')
        imshow(torchvision.utils.make_grid(images))
        plt.subplot(132)
        plt.title('Noise Image')
        imshow(torchvision.utils.make_grid(noise_images))
        plt.subplot(133)
        plt.title('Reconstruction')
        imshow(torchvision.utils.make_grid(
            outputs.view(images.size(0), num_channel, w, h).data
        ))
        plt.savefig('/content/drive/My Drive/images/filter_net_2.png')


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/My Drive/dataset/'